In [ ]:
%pip install sconce
%pip show sconce

import torch
import torchvision
import torchvision.transforms as transforms
import os
import torch.optim as optim
import numpy as np
import random

import copy
import torch.optim as optim
# import timm


import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List

import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
# from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm

# from torchprofile import profile_macs

assert torch.cuda.is_available(), \
"The current runtime does not have CUDA support." \
"Please go to menu bar (Runtime - Change runtime type) and select GPU"




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.1/153.1 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━

Name: sconce
Version: 0.57
Summary: sconce: torch pipeliner  
Home-page: https://github.com/satabios/sconce
Author: Sathyaprakash Narayanan
Author-email: Sathyaprakash Narayanan <snaray17@ucsc.edu>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, charset-normalizer, cmake, contourpy, cycler, fast-pytorch-kmeans, filelock, fonttools, idna, ipdb, Jinja2, kiwisolver, lit, MarkupSafe, matplotlib, mpmath, networkx, numpy, packaging, Pillow, pyparsing, python-dateutil, requests, six, snntorch, sympy, torch, torchprofile, torchvision, tqdm, transformers, typing-extensions, urllib3
Required-by: 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model_path = "drive/MyDrive/Efficientml/Efficientml.ai/pre-trained_vgg.cifar.pretrained.pth"

Mounted at /content/drive


In [ ]:

image_size = 32
transforms = {
    "train": transforms.Compose([
        RandomCrop(image_size, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
    ]),
    "test": ToTensor(),
}
dataset = {}
for split in ["train", "test"]:

    dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
    )

dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=512,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
  )

class VGG(nn.Module):
  ARCH = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

  def __init__(self) -> None:
    super().__init__()

    layers = []
    counts = defaultdict(int)

    def add(name: str, layer: nn.Module) -> None:
      layers.append((f"{name}{counts[name]}", layer))
      counts[name] += 1

    in_channels = 3
    for x in self.ARCH:
      if x != 'M':
        # conv-bn-relu
        add("conv", nn.Conv2d(in_channels, x, 3, padding=1, bias=False))
        add("bn", nn.BatchNorm2d(x))
        add("relu", nn.ReLU(True))
        in_channels = x
      else:
        # maxpool
        add("pool", nn.MaxPool2d(2))

    self.backbone = nn.Sequential(OrderedDict(layers))
    self.classifier = nn.Linear(512, 10)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # backbone: [N, 3, 32, 32] => [N, 512, 2, 2]
    x = self.backbone(x)

    # avgpool: [N, 512, 2, 2] => [N, 512]
    x = x.mean([2, 3])

    # classifier: [N, 512] => [N, 10]
    x = self.classifier(x)
    return x



#load the pretrained model

model = VGG().cuda()
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['state_dict'])



100%|██████████| 170498071/170498071 [00:05<00:00, 30173634.13it/s]


Extracting data/cifar10/cifar-10-python.tar.gz to data/cifar10
Files already downloaded and verified


<All keys matched successfully>

In [ ]:
from sconce import sconce


sconces = sconce()
sconces.model= model
sconces.criterion = nn.CrossEntropyLoss() # Loss
sconces.optimizer= optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
sconces.dataloader = dataloader
sconces.epochs = 1 #Number of time we iterate over the data
sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sconces.experiment_name = "vgg-gmp"

sconces.prune_mode = "GMP" # Supports Automated Pruning Ratio Detection

sconces.compress()


Dense_model_size model after sensitivity size=35.20 MiB


test:   0%|          | 0/20 [00:00<?, ?it/s]

Original Model Validation Accuracy: 93.13627254509018 %
Granular-Magnitude Pruning


test:   0%|          | 0/20 [00:00<?, ?it/s]

test:   0%|          | 0/20 [00:00<?, ?it/s]

...

train:   0%|          | 0/98 [00:00<?, ?it/s]

test:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch:5 Train Loss: 0.00000 Validation Accuracy: 93.08617


test:   0%|          | 0/20 [00:00<?, ?it/s]

/n
                Original        Pruned          Reduction Ratio
Latency (ms)    19400.0         19400.0         1.0            
MACs (M)        606             606             1.0            
Param (M)       9.23            9.23            1.0            
Fine-Tuned Sparse model has size=24.54 MiB = 69.72% of Original model size
Fine-Tuned Pruned Model Validation Accuracy: 93.08617234468937
